In [1]:
import functools
import geopy
from imposm.parser import OSMParser
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
import requests
import scipy as sp
import rtree
import seaborn as sb
from scipy import signal
# import shapely
import shapely.geometry
%pylab inline

import data_munging

Populating the interactive namespace from numpy and matplotlib


/home/zblan/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
rides, readings = data_munging.read_raw_data()
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

In [3]:
readings.columns

Index([u'id', u'created_at', u'updated_at', u'start_lat', u'start_lon',
       u'end_lat', u'end_lon', u'acceleration_x', u'acceleration_y',
       u'acceleration_z', u'angle_x', u'angle_y', u'angle_z', u'ride_id',
       u'start_time', u'end_time', u'mean_g', u'num_accel_x', u'abs_mean_x',
       u'std_x', u'num_accel_y', u'abs_mean_y', u'std_y', u'num_accel_z',
       u'abs_mean_z', u'std_z', u'std_total', u'duration', u'gps_dist',
       u'num_accel_total', u'abs_mean_total', u'gps_speed', u'gps_mph',
       u'total_readings', u'start_datetime', u'end_datetime',
       u'abs_mean_over_speed', u'start_x', u'start_y', u'end_x', u'end_y'],
      dtype='object')

In [4]:
readings.loc[0, ['num_accel_x', 'num_accel_y', 'num_accel_z', 'num_accel_total']]


num_accel_x        [0.0809326171875, 0.171295166016, 0.3459625244...
num_accel_y        [-0.680587768555, -0.698516845703, -0.59448242...
num_accel_z        [-0.207244873047, -0.503921508789, -1.33213806...
num_accel_total    [0.716030890839, 0.878182614638, 1.49923021354...
Name: 0, dtype: object

In [5]:
print rides.shape
print readings.shape
n, p = readings.shape

(106, 7)
(41773, 41)


In [6]:
nearest_request = 'http://127.0.0.1:5000/nearest?loc={0},{1}'
map_request = 'http://127.0.0.1:5000/match?loc={0},{1}&t={2}&loc={3},{4}&t={5}'

In [14]:
for snapped_var in ['snapped_' + var for var in ['start_lat', 'start_lon', 'end_lat', 'end_lon']]:
    readings[snapped_var] = 0
a_reading = readings.loc[0, :]

In [23]:
test_map_request = map_request.format(a_reading['start_lat'],
                                      a_reading['start_lon'], 
                                      0,
                                      a_reading['end_lat'],
                                      a_reading['end_lon'],
                                      1)
# This does not work because OSRM does not accept floats as times. 
# test_map_request = map_request.format(*tuple(a_reading[['start_lat', 'start_lon', 'start_time',
#                                                 'end_lat', 'end_lon', 'end_time']]))

In [26]:
osrm_response = requests.get(test_map_request).json()
osrm_response['matchings'][0]['matched_points']

[[41.896751, -87.634207], [41.89671, -87.634205]]

In [ ]:
total_calls = 0
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']])+9
    print i
    print data 
    osrm_request = nearest_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    total_calls += 1
    if total_calls > 100:
        break

In [ ]:
total_calls = 0
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
    print i
    print data 
    osrm_request = map_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    total_calls += 1
    if total_calls > 100:
        break

In [ ]:
for i, reading in readings.iterrows():
    data = tuple(reading[['start_lat', 'start_lon']]) + (0,) + tuple(reading[['end_lat', 'end_lon']]) + (1,)   
    osrm_request = map_request.format(*data)
    osrm_response = requests.get(osrm_request).json()
    print osrm_response
    readings.loc[i, ['snapped_start_lat', 'snapped_start_lon']] = osrm_response['matchings'][0]['matched_points'][0]
    break